# Neo4j

## Conexión con Cassandra y borrado de datos

In [1]:
%load_ext cypher

In [2]:
from py2neo import Graph, Node, Relationship

graph = Graph()

In [3]:
# Borrado de todos los nodos y relaciones
graph.delete_all()

## Carga de datos en Pandas

A partir del fichero Excel desnormalizamos los datos

In [11]:
import pandas as pd
df_mov = pd.read_excel("../../data/black.xlsx", sheetname= "Movimientos")
df_miembros = pd.read_excel("../../data/black.xlsx", sheetname= "Miembros")
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 0 to 7623
Data columns (total 11 columns):
id_miembro            7624 non-null int64
fecha                 7624 non-null datetime64[ns]
minuto                7624 non-null int64
hora                  7624 non-null int64
importe               7624 non-null float64
comercio              6864 non-null object
actividad_completa    7622 non-null object
actividad             7622 non-null object
nombre                7624 non-null object
funcion               7624 non-null object
organizacion          6056 non-null object
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 714.8+ KB


## Carga de datos en Neo4j

La carga se realizará en Neo a partir de la lista de datos en formato JSON, generados con Pandas

In [12]:
import json

json_string = df.to_json(orient = 'records')
json_list = json.loads(json_string)

En Neo se van a crear 3 tipos de nodos y 2 tipos de relaciones

<br><br> 

<img src="images/Modelo%20Neo4j.png",width=600,height=500>

<br><br> 

Adicionalmente se van a guardar los siguientes atributos:

**Nodo Comercio**
- Nombre del comercio
- Actividad a la que se dedica
- Categoría de actividad (Podríamos haber creado un tipo de nodo independente)

**Nodo Persona**
- Nombre de la persona

**Nodo Organización**
- Nombre de la organización

**Nodo Movimiento**
- Fecha en formato UNIX Timestamp (el mismo formato que está en el JSON)
- hora y minuto de la compra
- Importe de la compra

Respecto a las relaciones, vamos a almacenar los siguientes atributos:

**Relación PERTECENE**
- Función que hace la persona en una organización

Creamos los nodos y relaciones en Neo4j simplemente iterando sobre la lista de JSONs.

In [6]:
import datetime

for movimiento_json in json_list:
    # Nodos
    persona = Node("Persona", nombre=movimiento_json["nombre"])
    comercio = Node("Comercio", nombre=movimiento_json["comercio"], 
                                actividad = movimiento_json["actividad"],
                                actividad_completa = movimiento_json["actividad_completa"])
    organizacion = Node("Organizacion", nombre = movimiento_json["organizacion"])
    movimiento = Node("Movimiento",
            fecha = movimiento_json['fecha'],
            hora =  movimiento_json["hora"],
            minuto =  movimiento_json["minuto"],
            importe = movimiento_json["importe"])
    
        
    # Relaciones  
    rel_persona_organizacion = Relationship(persona, "PERTENECE", organizacion,
                                           funcion = movimiento_json["funcion"])
    rel_persona_movimiento = Relationship(persona, "REALIZA", movimiento)
    rel_movimiento_comercio = Relationship(movimiento, "OCURRE", comercio)
       
    graph.merge(persona | comercio | organizacion | movimiento | 
                rel_persona_organizacion | rel_persona_movimiento | rel_movimiento_comercio)


<br><br> 

<img src="images/Resultado%20Neo4j.png",width=800,height=600>

<br><br> 

### Comprobaciones previas ...

Número de movimientos ...

In [7]:
%%cypher
MATCH (:Movimiento)
RETURN count(*)

1 rows affected.


count(*)
7621


Número de personas ...

In [8]:
%%cypher
MATCH (:Persona)
RETURN count(*)

1 rows affected.


count(*)
83


Número de comercios

In [9]:
%%cypher
MATCH (:Comercio)
RETURN count(*)

1 rows affected.


count(*)
3283


Número de organizaciones

In [10]:
%%cypher
MATCH (:Organizacion)
RETURN count(*)

1 rows affected.


count(*)
10


All good!

## Funciones de utilidad para la consulta de datos

Neo4j, en su versión actual, no puede guardar datos de tipo fecha en los atributos, por lo que se almacena en formato TIMESTAMP de Unix (el formato original).  

La siguiente función de utilidad convierte una columna de una DataFrame en formato TIMESTAMP a Fecha

In [13]:
def transform_date(df, column_name):
    if column_name in df.columns:
        # Las fechas están en formato UNIX TIMESTAMP. Las volvemos a convertir a formato Date...
        df[column_name] = pd.to_datetime(df[column_name], unit = 'ms')
        
    return df

Función de utilidad para convertir los resultados devueltos por la base de datos en un DataFrame de Pandas

In [14]:
def get_dataframe(data):
    df = pd.DataFrame(list(data), columns = data.keys())
        
    return df

## Consulta a Neo4j

### Los 10 movimientos mas caros

Para obtener las lista de personas que mas han gastado simplemente relacionamos los 3 tipos de nodos y devolvemos los campos que nos interesan

In [16]:
%%cypher
MATCH (persona:Persona)-[:REALIZA]-(movimiento:Movimiento)-[:OCURRE]-(comercio:Comercio)
RETURN persona.nombre, comercio.actividad_completa, movimiento.importe
ORDER BY movimiento.importe DESC
LIMIT 10

10 rows affected.


persona.nombre,comercio.actividad_completa,movimiento.importe
Ildefonso José Sánchez Barcoj,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),11000.0
Ildefonso José Sánchez Barcoj,EL CORTE INGLES,6593.2
Matías Amat Roca,AGENCIAS DE VIAJES,6519.12
María Carmen Cafranga Cavestany,V.DIST.VIAJES Y TRANSPORTE DE VIAJEROS,5500.0
Ildefonso José Sánchez Barcoj,AGENCIAS DE VIAJES,5283.33
Estanislao Rodríguez-Ponga Salamanca,EL CORTE INGLES,5000.0
Ricardo Morado Iglesias,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),5000.0
Ramón Martínez Vilches,AGENCIAS DE VIAJES,4955.0
Ricardo Morado Iglesias,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),4500.0
Ildefonso José Sánchez Barcoj,EL CORTE INGLES,4320.5


Utilizamos la función ** graph.run()** para obtener un ResultSet de datos, que es transformada en un Dataframe de Pandas

In [17]:
query = """
MATCH (persona:Persona)-[:REALIZA]-(movimiento:Movimiento)-[:OCURRE]-(comercio:Comercio)
RETURN persona.nombre, comercio.actividad_completa, movimiento.importe
ORDER BY movimiento.importe DESC
LIMIT 10
""" 

data = graph.run(query)
df = get_dataframe(data)
df = transform_date(df, 'compra.fecha')
df

,persona.nombre,comercio.actividad_completa,movimiento.importe
0,Ildefonso José Sánchez Barcoj,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),11000.00
1,Ildefonso José Sánchez Barcoj,EL CORTE INGLES,6593.20
2,Matías Amat Roca,AGENCIAS DE VIAJES,6519.12
3,María Carmen Cafranga Cavestany,V.DIST.VIAJES Y TRANSPORTE DE VIAJEROS,5500.00
4,Ildefonso José Sánchez Barcoj,AGENCIAS DE VIAJES,5283.33
5,Estanislao Rodríguez-Ponga Salamanca,EL CORTE INGLES,5000.00
6,Ricardo Morado Iglesias,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),5000.00
7,Ramón Martínez Vilches,AGENCIAS DE VIAJES,4955.00
8,Ricardo Morado Iglesias,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),4500.00
9,Ildefonso José Sánchez Barcoj,EL CORTE INGLES,4320.50


La misma query que el caso anterior pero utilizando un método distinto.  
El fin es el mismo: Una DataFrame de Pandas

In [18]:
df = %cypher MATCH (persona:Persona)-[:REALIZA]-(movimiento:Movimiento)-[:OCURRE]-(comercio:Comercio) \
             RETURN persona.nombre, comercio.actividad_completa, movimiento.importe \
             ORDER BY movimiento.importe DESC \
             LIMIT 10
            
df.get_dataframe()

10 rows affected.


,persona.nombre,comercio.actividad_completa,movimiento.importe
0,Ildefonso José Sánchez Barcoj,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),11000.00
1,Ildefonso José Sánchez Barcoj,EL CORTE INGLES,6593.20
2,Matías Amat Roca,AGENCIAS DE VIAJES,6519.12
3,María Carmen Cafranga Cavestany,V.DIST.VIAJES Y TRANSPORTE DE VIAJEROS,5500.00
4,Ildefonso José Sánchez Barcoj,AGENCIAS DE VIAJES,5283.33
5,Estanislao Rodríguez-Ponga Salamanca,EL CORTE INGLES,5000.00
6,Ricardo Morado Iglesias,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),5000.00
7,Ramón Martínez Vilches,AGENCIAS DE VIAJES,4955.00
8,Ricardo Morado Iglesias,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),4500.00
9,Ildefonso José Sánchez Barcoj,EL CORTE INGLES,4320.50


### Importes de una persona agrupados por actividad

En este caso se realiza una agrupación de los datos. Observa que Neo4j lo realiza automáticamente al utilizar una función de agrupación de datos como **SUM()**

In [19]:
%%cypher
MATCH (persona:Persona)-[:REALIZA]-(movimiento:Movimiento)-[:OCURRE]-(comercio:Comercio)
WHERE comercio.actividad = 'HOGAR'
RETURN persona.nombre, comercio.actividad_completa, SUM(movimiento.importe) as importe
ORDER BY importe DESC
LIMIT 10


10 rows affected.


persona.nombre,comercio.actividad_completa,importe
Mariano Pérez Claver,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",2210.41
Miguel Ángel Araujo Serrano,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",2150.0
Carlos María Martínez Martínez,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",2031.6
Ricardo Romero de Tejada y Picatoste,"TAPICERIAS,ALFOMBRAS",1339.94
Rafael Spottorno Díaz Caro,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",1283.0
Rafael Spottorno Díaz Caro,MIRO ESTABLECIMIENTOS,1198.0
Ricardo Romero de Tejada y Picatoste,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",1108.0
Ricardo Romero de Tejada y Picatoste,FLORES Y PLANTAS,1000.0
Alejandro Couceiro Ojeda,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",980.95
Carlos María Martínez Martínez,FLORES Y PLANTAS,962.6


In [23]:
query = """
MATCH (organizacion:Organizacion)-[:PERTENECE]-(persona:Persona)-[:REALIZA]-(movimiento:Movimiento)-[:OCURRE]-(comercio:Comercio)
WHERE persona.nombre = {name}
RETURN persona.nombre, comercio.actividad, SUM(movimiento.importe) as importe
ORDER BY importe DESC
"""

data = graph.run(query, name='Mariano Pérez Claver')
get_dataframe(data)

,persona.nombre,comercio.actividad,importe
0,Mariano Pérez Claver,CA$H,4200.00
1,Mariano Pérez Claver,AVION,3596.71
2,Mariano Pérez Claver,RESTAURANTE,2992.66
3,Mariano Pérez Claver,VIAJE,2959.67
4,Mariano Pérez Claver,HOGAR,2383.18
5,Mariano Pérez Claver,ROPA,2177.81
6,Mariano Pérez Claver,BARCO,1909.69
7,Mariano Pérez Claver,COMPRA BIENES,1554.10
8,Mariano Pérez Claver,SUPERMERCADO,1295.54
9,Mariano Pérez Claver,COCHE,584.14
